In [11]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [12]:
transformer = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True,
                                        transform=transformer, download=True)
testset = torchvision.datasets.CIFAR10(root="./data", train=False,
                                       transform=transformer, download=True)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

In [13]:
print(len(trainset))
print(len(testset))

50000
10000


In [14]:
trainset[0][0].shape # 3 цветовых канала, сами картинки размером 32 на 32 пикселя

torch.Size([3, 32, 32])

In [15]:
class MyOwnNet(nn.Module):
    def __init__(self):
      super(MyOwnNet, self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3)
      self.bn1 = nn.BatchNorm2d(12)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
      self.conv2 = nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3)
      self.bn2 = nn.BatchNorm2d(16)
      self.conv3 = nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3)
      self.bn3 = nn.BatchNorm2d(24)

      self.dropout = nn.Dropout(p=0.3)
      self._to_linear = None
      self._get_conv_size(torch.randn(1, 3, 32, 32))

      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(self._to_linear, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

    def _get_conv_size(self, x):
          x = self.pool(F.relu(self.bn1(self.conv1(x))))
          x = self.pool(F.relu(self.bn2(self.conv2(x))))
          x = self.pool(F.relu(self.bn3(self.conv3(x))))
          self._to_linear = x.flatten(1).shape[1]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # Последний слой без ФА, т.к. в Лосс уже встроен софтмакс
        return x

In [17]:
model = MyOwnNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [18]:
dataloaders = {"train": trainloader, "test": testloader}

In [19]:
model.train()
epochs = 100
accuracy = {"train": [], "test": []}
for epoch in tqdm(range(epochs)):
  for action, dataloader in dataloaders.items():
    epoch_correct = 0
    epoch_all = 0
    for X_batch, y_batch in dataloader:
      X_batch = X_batch.to(device)
      y_batch = y_batch.to(device)
      if action == "train":
        model.train()
        optimizer.zero_grad()
        logits = model(X_batch)
      else:
        model.eval()
        with torch.no_grad():
          logits = model(X_batch)
      y_pred = logits.argmax(-1)
      correct_preds = (y_batch == y_pred).sum()
      epoch_correct += correct_preds.item()
      epoch_all += y_batch.shape[0]
      if action == "train":
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
    if action == "train":
      print(f"Epoch №{epoch + 1}")
    print(f"Loader: {action}, accuracy: {epoch_correct/epoch_all}")
    accuracy[action].append(epoch_correct/epoch_all)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch №1
Loader: train, accuracy: 0.2977


  1%|          | 1/100 [00:28<46:46, 28.35s/it]

Loader: test, accuracy: 0.39634
Epoch №2
Loader: train, accuracy: 0.4111


  2%|▏         | 2/100 [00:53<43:07, 26.40s/it]

...

Loader: test, accuracy: 0.73854
Epoch №99
Loader: train, accuracy: 0.66874


 99%|█████████▉| 99/100 [39:48<00:24, 24.08s/it]

Loader: test, accuracy: 0.74266
Epoch №100
Loader: train, accuracy: 0.6679


100%|██████████| 100/100 [40:12<00:00, 24.12s/it]

Loader: test, accuracy: 0.73566


In [20]:
torch.save(model.state_dict(), 'CIFAR10_v2_100_EPOCHS.pth')